In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot

In [ ]:
mechanism = StrandbeestRobot.mechanism(floating=false, add_flat_ground=false);

In [ ]:
gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()));

In [ ]:
#NBSKIP
open(gui);

In [ ]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)
copyto!(gui.visualizer, state)

In [ ]:
# Simulate the robot
control! = let I = velocity_range(state, findjoint(mechanism, "joint_crossbar_crank"))
    function (τ, t, state)
        τ[I] .= -10.0 .* (velocity(state)[I] .- 3.0)
    end
end
dynamics = Dynamics(mechanism, control!)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)